<a href="https://colab.research.google.com/github/ashikita/pdf_adjustment/blob/main/pdf_encrypt_copy_restrict.py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# -*- coding: utf-8 -*-
"""
PDFの本文コピーを制限する

"""

# === 設定ブロック ===
USER_PASSWORD = "your_user_password"   # 閲覧時に必要なパスワード
OWNER_PASSWORD = "your_owner_password" # すべての権限を持つパスワード（未設定なら USER_PASSWORD と同じにする）
ALGORITHM_PREFS = ("AES-256", "AES-128", "RC4-128")  # 利用可能なものがあれば順に使う
ALLOW_ACCESSIBILITY = True  # スクリーンリーダー用のテキスト抽出を許可する場合は True

# === ここから実装 ===
import os
import sys
import time
import glob

# ライブラリ検出（pypdf → PyPDF2 の順に試行）
PdfReader = PdfWriter = Permissions = None
lib_name = None
try:
    import pypdf  # 新名称
    from pypdf import PdfReader, PdfWriter
    try:
        from pypdf import Permissions  # pypdf>=3 目安
    except Exception:
        Permissions = None
    lib_name = "pypdf"
except Exception:
    try:
        import PyPDF2  # 旧名称
        from PyPDF2 import PdfReader, PdfWriter
        try:
            from PyPDF2.constants import Permissions  # PyPDF2>=2.12 目安
        except Exception:
            Permissions = None
        lib_name = "PyPDF2"
    except Exception as e:
        print("ERROR: pypdf / PyPDF2 が利用できません。ライブラリをインストールしてください。", file=sys.stderr)
        raise

# 許可する権限セット（印刷のみ許可、本文コピーは禁止）
allowed_permissions = None
if Permissions is not None:
    allowed_permissions = {getattr(Permissions, "PRINTING")}
    # アクセシビリティ（スクリーンリーダー）許可
    if ALLOW_ACCESSIBILITY and hasattr(Permissions, "ACCESSIBILITY"):
        allowed_permissions.add(getattr(Permissions, "ACCESSIBILITY"))

def encrypt_with_best_algorithm(writer, user_pwd, owner_pwd, perms):
    """
    ライブラリやバージョン差異に合わせて encrypt 呼び出しを試行。
    ・permissions を明示し、コピー不可・印刷可を実現（対応版のみ）
    ・アルゴリズムは優先リストで利用可のものを採用
    """
    # owner 未設定なら user を流用
    if not owner_pwd:
        owner_pwd = user_pwd

    # permissions 引数あり + algorithm 指定のパターン（pypdf 新しめ）
    for algo in ALGORITHM_PREFS:
        try:
            if perms is not None:
                writer.encrypt(
                    user_password=user_pwd,
                    owner_password=owner_pwd,
                    permissions=perms,
                    algorithm=algo,
                )
            else:
                writer.encrypt(
                    user_password=user_pwd,
                    owner_password=owner_pwd,
                    algorithm=algo,
                )
            return
        except TypeError:
            # キーワード名が異なる／未対応 → 次の試行へ
            pass
        except Exception:
            # アルゴリズム非対応など → 次の試行へ
            pass

    # algorithm キーワード無しのパターン
    try:
        if perms is not None:
            writer.encrypt(
                user_password=user_pwd,
                owner_password=owner_pwd,
                permissions=perms,
            )
        else:
            writer.encrypt(
                user_password=user_pwd,
                owner_password=owner_pwd,
            )
        return
    except TypeError:
        pass

    # user_password ではなく password を使う実装向け
    try:
        if perms is not None:
            writer.encrypt(
                password=user_pwd,
                owner_password=owner_pwd,
                permissions=perms,
            )
        else:
            writer.encrypt(
                password=user_pwd,
                owner_password=owner_pwd,
            )
        return
    except TypeError:
        pass

    # 旧 PyPDF2 互換（permissions を受け取らない場合あり・128bit固定など）
    try:
        # use_128bit が使える版
        writer.encrypt(user_pwd, owner_pwd, use_128bit=True)
        return
    except TypeError:
        # 最後のフォールバック（仕様不明瞭だが試す）
        writer.encrypt(user_pwd, owner_pwd)

def process_pdf(in_path, out_path):
    # 読み込み（暗号化済み PDF への対処）
    reader = PdfReader(in_path)
    if getattr(reader, "is_encrypted", False):
        # 空パスワードで開けるケースの回避試行（開けない場合は例外で落とす）
        try:
            # pypdf: decrypt("") は不要なことが多いが互換のため保持
            if hasattr(reader, "decrypt"):
                reader.decrypt("")
        except Exception:
            pass

    writer = PdfWriter()

    # ページコピー
    for page in reader.pages:
        writer.add_page(page)

    # メタデータの継承（失敗しても処理継続）
    try:
        meta = getattr(reader, "metadata", None)
        if meta:
            # pypdf/PyPDF2 は dict 互換を期待
            writer.add_metadata({k: v for k, v in meta.items() if v is not None})
    except Exception:
        pass

    # 暗号化設定（コピー不可・印刷可）
    encrypt_with_best_algorithm(
        writer,
        USER_PASSWORD,
        OWNER_PASSWORD or USER_PASSWORD,
        allowed_permissions,
    )

    # 書き出し
    with open(out_path, "wb") as f:
        writer.write(f)

def main():
    os.makedirs("output", exist_ok=True)

    # カレント直下のPDF（output ディレクトリは対象外）
    candidates = []
    for pattern in ("*.pdf", "*.PDF", "*.Pdf", "*.pDf", "*.pdF"):
        candidates.extend(glob.glob(pattern))
    candidates = [p for p in sorted(set(candidates)) if not p.startswith("output"+os.sep)]

    if not candidates:
        print("INFO: 対象となる PDF が見つかりませんでした。")
        return

    for pdf_path in candidates:
        start = time.perf_counter()
        status = "SUCCESS"
        err_msg = ""
        try:
            out_path = os.path.join("output", os.path.basename(pdf_path))
            process_pdf(pdf_path, out_path)
        except Exception as e:
            status = "ERROR"
            err_msg = str(e)
        elapsed = time.perf_counter() - start

        # 要求仕様：ファイル名、成功・エラーのステータス、かかった時間
        if status == "SUCCESS":
            print(f"{os.path.basename(pdf_path)},SUCCESS,{elapsed:.3f}s")
        else:
            print(f"{os.path.basename(pdf_path)},ERROR,{elapsed:.3f}s,{err_msg}")

if __name__ == "__main__":
    main()
